In [ ]:
!pip install dash

In [1]:
import pandas as pd
import numpy as np
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output,State
import plotly.graph_objects as go
print('Your current dash board version is:' + dash.__version__)

Your current dash board version is:2.6.0


In [2]:
# import local CSV file as a dataframe
df_input_large=pd.read_csv('C:/Users/kriti/ADS-COVID19/data/processed/COVID_final_set2.csv',sep=';')

In [ ]:
# for plotting
fig = go.Figure()
# for dashboard development
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Data Science Project From TU_KL on COVID-19 Dataset-Part 1
    * Goal of this project is to learn data science by applying a cross-industry standard process. 
      The default layout contains the confirmed infected cases in the logarithmic-scale format for options (1 & 2); Approximated doubling rate
      for options (3 & 4) on the Y-axis and with the Timeline in Days on the X-axis (Upto the latest in July 2022).
    ### The first dropdown menu enables selection of one or multiple  countries for visualization. The seconds dropdown menu contains four options:
        1. The ‘Timeline Confirmed’ represents confirmed infected cases in each country along the timeline.
        2. The ‘Timeline Confirmed Filtered’ represents filtered (after applying sav-gol filter) confirmed infected cases in the same countries along the timeline.
        3. The ‘Timeline Doubling Rate’ represents calculated doubling rate on the infected cases along the timeline from the 1st option.
        4. The ‘Timeline Doubling Rate Filtered’ represents calculated doubling rate on the infected cases along the timeline from the 2nd option.
    '''),

    dcc.Markdown('''
    ## Select the Country for visualization
    '''),

    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['Germany','India'], # which are pre-selected in default layout
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 deaths or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline deaths ', 'value': 'deaths'},
        {'label': 'Timeline deaths Filtered', 'value': 'deaths_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',multi=False),dcc.Graph(figure=fig, id='main_window_slope')])

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure_layout(country_list,show_doubling):
    if 'DR' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
          }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed Infected People (From Johns Hopkins csse, log-scale)'}
    traces = []
    for each in country_list:
        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','deaths','deaths_filtered','deaths_DR','deaths_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','deaths','deaths_filtered','deaths_DR','deaths_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()

        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',opacity=1.0,name=each))

    return {
            'data': traces,
            'layout': dict (
                width=1000,height=650,
                xaxis={'title':'Timeline in the days','tickangle':-45,'nticks':20,
                        'tickfont':dict(size=14,color="#0c6887"),},yaxis=my_yaxis)}

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
